<a href="https://colab.research.google.com/github/Valerie231/ML_homeworks/blob/main/HW_2_9_%D0%9D%D0%B5%D0%B7%D0%B1%D0%B0%D0%BB%D0%B0%D0%BD%D1%81%D0%BE%D0%B2%D0%B0%D0%BD%D0%B0_%D0%B1%D0%B0%D0%B3%D0%B0%D1%82%D0%BE%D0%BA%D0%BB%D0%B0%D1%81%D0%BE%D0%B2%D0%B0_%D0%BA%D0%BB%D0%B0%D1%81%D0%B8%D1%84%D1%96%D0%BA%D0%B0%D1%86%D1%96%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [72]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.metrics import roc_curve, auc
import operator
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import classification_report, precision_score, recall_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from imblearn.over_sampling import SMOTENC

In [3]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/Colab Notebooks/

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks


In [4]:
df = pd.read_csv('customer_segmentation_train.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [6]:
df.isnull().sum()

ID                   0
Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       0
Family_Size        335
Var_1               76
Segmentation         0
dtype: int64

In [7]:
X = df.drop(columns=['Segmentation', 'ID'])
y = df['Segmentation']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=12)

In [9]:
numeric_cols = X_train.select_dtypes(exclude = 'object').columns
categorical_cols = X_train.select_dtypes(include = 'object').columns

In [10]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
])

In [11]:
categorical_transformer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

In [12]:
preprocess = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Create a pipeline that includes preprocessing and the model
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocess),
    ('classifier', OneVsRestClassifier(LogisticRegression(solver='liblinear')))
])

# Train the model
model_pipeline.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  Index(['Age', 'Work_Experience', 'Family_Size'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  Index(['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score',
       'Var_1'],
      dtype='object'))])),
                ('classifier',
                 OneVsRestClassifier(estimator=LogisticRegression(solver='liblinear')))])

In [13]:
model_pipeline.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  Index(['Age', 'Work_Experience', 'Family_Size'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  Index(['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score',
       'Var_1'],
      dtype='object'))])),
                ('classifier',
                 OneVsRestClassifier(estimator=LogisticRegression(solver='liblinear')))])

In [14]:
train_preds = model_pipeline.predict(X_train)
test_preds = model_pipeline.predict(X_test)

In [15]:
print(classification_report(y_train, train_preds))

              precision    recall  f1-score   support

           A       0.42      0.49      0.45      1573
           B       0.41      0.18      0.25      1490
           C       0.50      0.63      0.56      1565
           D       0.64      0.71      0.67      1826

    accuracy                           0.51      6454
   macro avg       0.49      0.50      0.48      6454
weighted avg       0.50      0.51      0.49      6454



In [16]:
print(classification_report(y_test, test_preds))

              precision    recall  f1-score   support

           A       0.44      0.51      0.47       399
           B       0.43      0.20      0.28       368
           C       0.53      0.65      0.58       405
           D       0.67      0.73      0.70       442

    accuracy                           0.54      1614
   macro avg       0.52      0.52      0.51      1614
weighted avg       0.52      0.54      0.52      1614



**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [48]:
from imblearn.over_sampling import SMOTE

# Perform random sampling
smote = SMOTE(random_state=0)

X_train_transformed = numeric_transformer.fit_transform(X_train[numeric_cols])
X_train_smote, y_train_smote = smote.fit_resample(X_train_transformed, y_train)

X_test_transformed = numeric_transformer.transform(X_test[numeric_cols])

In [54]:
smote_train_preds = model_pipeline.named_steps['classifier'].fit(X_train_smote, y_train_smote)

In [56]:
smote_train_preds = model_pipeline.named_steps['classifier'].predict(X_train_smote)
smote_test_preds = model_pipeline.named_steps['classifier'].predict(X_test_transformed)

In [58]:
print(classification_report(y_train_smote, smote_train_preds))

              precision    recall  f1-score   support

           A       0.32      0.30      0.31      1826
           B       0.28      0.13      0.18      1826
           C       0.38      0.40      0.39      1826
           D       0.46      0.72      0.56      1826

    accuracy                           0.39      7304
   macro avg       0.36      0.39      0.36      7304
weighted avg       0.36      0.39      0.36      7304



In [59]:
print(classification_report(y_test, smote_test_preds))

              precision    recall  f1-score   support

           A       0.36      0.35      0.36       399
           B       0.26      0.13      0.17       368
           C       0.41      0.42      0.42       405
           D       0.53      0.75      0.62       442

    accuracy                           0.43      1614
   macro avg       0.39      0.41      0.39      1614
weighted avg       0.40      0.43      0.40      1614



Using SMOTENC

In [61]:
X_train_transformed = model_pipeline.named_steps['preprocessor'].fit_transform(X_train)
X_test_transformed = model_pipeline.named_steps['preprocessor'].transform(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [70]:
categorical_indices = [X_train.columns.get_loc(col) for col in categorical_cols]

In [73]:
sm = SMOTENC(random_state=42, categorical_features=categorical_indices)
X_train_smotenc, y_train_smotenc = sm.fit_resample(X_train_transformed, y_train)

In [74]:
model_pipeline.named_steps['classifier'].fit(X_train_smotenc, y_train_smotenc)

smotenc_train_preds = model_pipeline.named_steps['classifier'].predict(X_train_smotenc)
smotenc_test_preds = model_pipeline.named_steps['classifier'].predict(X_test_transformed)

In [75]:
print(classification_report(y_train_smotenc, smotenc_train_preds))

              precision    recall  f1-score   support

           A       0.43      0.51      0.46      1826
           B       0.41      0.24      0.30      1826
           C       0.51      0.60      0.55      1826
           D       0.63      0.66      0.65      1826

    accuracy                           0.50      7304
   macro avg       0.49      0.50      0.49      7304
weighted avg       0.49      0.50      0.49      7304



In [76]:
print(classification_report(y_test, smotenc_test_preds))

              precision    recall  f1-score   support

           A       0.45      0.52      0.48       399
           B       0.41      0.25      0.31       368
           C       0.54      0.63      0.58       405
           D       0.69      0.69      0.69       442

    accuracy                           0.53      1614
   macro avg       0.52      0.52      0.52      1614
weighted avg       0.53      0.53      0.52      1614



SMOTE_Tomek

In [80]:
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks

In [83]:
smote_tomek = SMOTETomek(random_state=12)
tl = TomekLinks()

In [84]:
X_train_transformed = numeric_transformer.fit_transform(X_train[numeric_cols])
X_train_smtk, y_train_smtk = smote_tomek.fit_resample(X_train_transformed, y_train)

X_test_transformed = numeric_transformer.transform(X_test[numeric_cols])

In [85]:
model_pipeline.named_steps['classifier'].fit(X_train_smtk, y_train_smtk)

OneVsRestClassifier(estimator=LogisticRegression(solver='liblinear'))

In [86]:
smtk_train_preds = model_pipeline.named_steps['classifier'].predict(X_train_smtk)
smtk_test_preds = model_pipeline.named_steps['classifier'].predict(X_test_transformed)

In [87]:
print(classification_report(y_train_smtk, smtk_train_preds))

              precision    recall  f1-score   support

           A       0.32      0.28      0.30      1741
           B       0.28      0.12      0.17      1741
           C       0.38      0.43      0.40      1749
           D       0.47      0.73      0.57      1759

    accuracy                           0.39      6990
   macro avg       0.36      0.39      0.36      6990
weighted avg       0.36      0.39      0.36      6990



In [88]:
print(classification_report(y_test, smtk_test_preds))

              precision    recall  f1-score   support

           A       0.37      0.32      0.34       399
           B       0.27      0.13      0.17       368
           C       0.41      0.46      0.43       405
           D       0.52      0.76      0.62       442

    accuracy                           0.43      1614
   macro avg       0.39      0.42      0.39      1614
weighted avg       0.40      0.43      0.40      1614



**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

1. Кроки 1-2 в попередніх завданнях :)
2.
3. Обрала weighted average F1-Score
4. Найкращими виявились моделі на оригінальних та SMOTENC resampled даних, проте їні показники майже однакові, тому ресамплінг суттєво не змінив результату
5. Можливо причина в тому, що класи не були суттєво дизбалансовані, тому модель достатньо правильно визначала decision boundaries і до ресамплінгу.
